# Начало отчета

In [ ]:

# %%javascript
# IPython.notebook.kernel.execute('ipynb_file_name_ext = "' + IPython.notebook.notebook_name + '"')
# // получить имя файла не питона а юпитер ноут в переменную ipynb_file_name
# // работает только когда запускаем вручную 

# // или
 
# %%javascript
# IPython.notebook.kernel.execute(`ipynb_file_name_ext = '${window.document.getElementById("notebook_name").innerHTML}'`);
# // получить имя файла не питона а юпитер ноут в переменную ipynb_file_name
# // работает только когда запускаем вручную 

In [ ]:
# ipynb_file_name_ext

In [ ]:
# задаем имя файла если жаваскрипт отработал криво а он нифига не пашет (( только если руками запускать а если ран алл то все криво работает ((
ipynb_file_name = 'Среднесуточные складские остатки в разрезе индекса продаж'
# ipynb_file_name

In [ ]:
# magic
#################################
%matplotlib inline 
#################################

# run magic в юпитере! НЕ РАБОТАЕТ в .py файлах
# run files исполнить файлы магией тут
%run /srv/jhub/share/._Additions/import_my_modules.py

# %run /srv/jhub/share/._Additions/import_libs.py
# %run /srv/jhub/share/._Additions/prints_.py
# %run /srv/jhub/share/._Additions/my_script.py этот импортится внутри import_my_modules.py
# %run /srv/jhub/share/._Additions/Connecting_.py
# %run /srv/jhub/share/._Additions/Date_gen.py
# %run /srv/jhub/share/._Additions/xx.py

#################################
# еще вариант magic который !РАБОТАЕТ в .py файлах
# get_ipython().run_line_magic('matplotlib', 'inline')
# run files
# get_ipython().run_line_magic('run', '/srv/jhub/share/._Additions/import_libs.py')
# get_ipython().run_line_magic('run', '/srv/jhub/share/._Additions/Connecting_.py')
# get_ipython().run_line_magic('run', '/srv/jhub/share/._Additions/my_script.py')
# get_ipython().run_line_magic('run', '/srv/jhub/share/._Additions/Date_gen.py')
# get_ipython().run_line_magic('run', '/srv/jhub/share/._Additions/xx.py')
############################## End ##############################
# help(Prints_), print(JiraTaskPrint.__doc__)

## Жмем сочетание клавиш:  Alt-Ctrl-L (на англ. раскладке)

In [ ]:
# выводим спарсенные данные с жиры описание и тд....
task_link = '''
https://jira.e2e4.ru/browse/BA-\
380
'''
JiraTaskPrint().print_tasks(task_link)

In [ ]:
spl1 = '1000-01-01 00:00:00'
spl2 = '1000-01-02 00:00:00'
spl3 = '1000-01-03 00:00:00'


In [ ]:

sql_query_1 = ''' 
    WITH params AS (
    select
        /*jhub*/
            'DATE_START_replce'::date AS frm, 
            'DATE_END_replce'::date AS til,
            STORAGE_replce AS storage,
            CATEGORY_replce AS category,
            'BRAND_replce' as brand
            --DEL_IN_USE_replce as in_use
            --PRODUCTMANAGER_replce 
            --DEL_STORAGE_NSK_replce 

        /*confluence*/
    --        '\$from'::date AS frm,
    --        '\$til'::date AS til,
    --		\$storage AS storage,
    --		\$category AS category,
    --        '\$brand' AS brand

        /*tests*/
    --        '01.08.2022'::date AS frm, 
    --        '19.08.2022'::date AS til,
    --        0 AS storage,
    --        0 AS category,
    --        '' as brand
    )
	, items AS (
	    SELECT i.id, i.name, i."rank", i.id_category
	    FROM params _
	    CROSS JOIN "catalog".items i
		JOIN "catalog".categories c ON c.id = i.id_category
		LEFT JOIN "catalog".brands b ON b.id = i.id_brand
	    WHERE i.RANK IS NOT NULL
	        AND CASE WHEN _.category = 0 THEN TRUE ELSE i.id_category = _.category END
	        AND CASE WHEN _.brand = '' THEN TRUE ELSE b.name ilike _.brand end
	        
	--        and case when 1 = 1 then true else c.name not ilike 'уцененные товары%' and c.name not ilike 'внутреннее использование%' end
	        /*jhub*/
	        and case when DEL_IN_USE_replce = 1 then true else c.name not ilike 'уцененные товары%' and c.name not ilike 'внутреннее использование%' end
	
	)
	, categories AS (
	    select distinct c.id, c.name, i.rank, c.order, pu.name "suppliers_name"
	    FROM items i
	    JOIN "catalog".categories c ON c.id = i.id_category
	    left join public.users pu on c.id_product_manager = pu.id 
	    where 
	--	    case when 0 = 0 then true
	--	        else c.id_product_manager = 0 end
	--		and case when 1 = 1 then true 
	--			else c.name not ilike 'Уцененные товары%' and c.name not ilike 'Внутреннее использование%' end
	        /*jhub*/
		    case when PRODUCTMANAGER_replce = 0 then true
		        else c.id_product_manager = PRODUCTMANAGER_replce end
		    and case when DEL_IN_USE_replce = 1 then true 
				else c.name not ilike 'Уцененные товары%' and c.name not ilike 'Внутреннее использование%' end  
	)
	, sales AS (
	    SELECT
	        i.id_category,
	        i."rank",
			count(DISTINCT i.id) AS count,
			case when (count(s.id)::numeric / (1 + _.til - _.frm)) = 0 then null 
				else count(s.id)::numeric / (1 + _.til - _.frm) end AS quantity,
	        case when (sum(s."cost")::numeric / (1 + _.til - _.frm)) = 0 then null 
	        	else sum(s."cost")::numeric / (1 + _.til - _.frm) end AS sum
	    FROM params _
	    CROSS JOIN items i
		JOIN sales.items s ON s.id_catalog_item = i.id
		JOIN public.storage st ON st.id = s.id_storage
		WHERE s.sold_at BETWEEN _.frm AND _.til
			AND CASE WHEN _.storage = 0 THEN TRUE ELSE COALESCE(st.parentstorageid, st.id) = _.storage END
		GROUP BY _.frm, _.til, i.id_category, i."rank"
	)
	, stock AS (
	    SELECT
	        i.id_category,
	        i."rank",
			sum(h.stock_free
				+ h.stock_reserved
				+ h.relocation_free
				+ h.relocation_reserved
				+ h.arrivals_free
				+ h.arrivals_reserved
				+ h.transit_free
				+ h.transit_reserved)::numeric / (1 + _.til - _.frm) AS quantity,
	        sum(h.stock_free_sum
				+ h.stock_reserved_sum
				+ h.relocation_free_sum
				+ h.relocation_reserved_sum
				+ h.arrivals_free_sum
				+ h.arrivals_reserved_sum
				+ h.transit_free_sum
				+ h.transit_reserved_sum)::numeric / (1 + _.til - _.frm) AS sum,
	        sum(h.stock_free
				+ h.relocation_free
				+ h.arrivals_free
				+ h.transit_free
	        )::numeric / (1 + _.til - _.frm) AS quantity_free,
	        sum(h.stock_free_sum
				+ h.relocation_free_sum
				+ h.arrivals_free_sum
				+ h.transit_free_sum
	        )::numeric / (1 + _.til - _.frm) AS sum_free
	    FROM params _
	    CROSS JOIN items i
	    JOIN store.stock_history h ON h.id_reference_item = i.id
		JOIN public.storage st ON st.id = h.id_storage
		WHERE 
			h."date" BETWEEN _.frm AND _.til
			AND CASE WHEN _.storage = 0 THEN TRUE 
				ELSE COALESCE(st.parentstorageid, st.id) = _.storage end
				
	--		and case when 1 = 1 then true 
	--			else st.id not in ( 47 ) end
			/*jhub*/
			and case when DEL_STORAGE_NSK_replce = 0 then true 
				else st.id not in ( DEL_STORAGE_NSK_replce ) end
	
		GROUP BY _.frm, _.til, i.id_category, i."rank"
	)
	
	, EA as (
	select
	    c.suppliers_name "Продакт",
	--    c.name "Категория",
	    c.RANK "Индекс продаж",
	    sum(sales.count) "Продажи уникальных артикулов, шт",
	    sum(round(sales.quantity, 2)) "СС Продажи, шт",
	    sum(sales.sum::int) "СС Продажи, руб (себестоимость)",
	    sum(round(stock.quantity, 2)) "СС Остатки, шт",
	    sum(stock.sum::int) "СС Остатки, руб (себестоимость)",
	    sum(round(stock.quantity_free, 2)) "СС Остатки-свободный, шт",
	    sum(stock.sum_free::int) "СС Остатки-свободный, руб (себестоимость)",
	    CASE WHEN sum(sales.quantity)=0 THEN NULL ELSE sum(round(stock.quantity / sales.quantity)) END AS "Оборачиваемость (шт/шт)",
	    CASE WHEN sum(sales.sum)=0 THEN NULL ELSE sum(round(stock.sum / sales.sum)) END AS "Оборачиваемость (руб/руб)"
	FROM categories c
	LEFT JOIN sales ON sales.id_category = c.id AND sales.RANK = c.rank
	LEFT JOIN stock ON stock.id_category = c.id AND stock.RANK = c.RANK
	WHERE 
		GREATEST(sales.quantity, sales.sum, stock.quantity, stock.sum) > 0
	group by  
		c.suppliers_name, c.rank --c.suppliers_name,
	ORDER BY 
		c.rank --c."order", c.name, 
	)
	
    select * from EA

    union all
    select 
        'ИТОГО',
        'ИТОГО', 
        sum("Продажи уникальных артикулов, шт") "Продажи уникальных артикулов, шт"
        , sum("СС Продажи, шт") "СС Продажи, шт"
        , sum("СС Продажи, руб (себестоимость)") "СС Продажи, руб (себестоимость)"
        , sum("СС Остатки, шт") "СС Остатки, шт"
        , sum("СС Остатки, руб (себестоимость)") "СС Остатки, руб (себестоимость)"
        , sum("СС Остатки-свободный, шт") "СС Остатки-свободный, шт"
        , sum("СС Остатки-свободный, руб (себесто") "СС Остатки-свободный, руб (себесто"
        , sum("Оборачиваемость (шт/шт)") "Оборачиваемость (шт/шт)"
        , sum("Оборачиваемость (руб/руб)") "Оборачиваемость (руб/руб)"
    from EA

    --union all
    --select 
    --	'ИТОГО',
    --	'конфа', 13863,	3677.25,	10614289,	359909.52,	667188106,	319112.76,	523616688,	189068,	261366

    /*
    --union all
    --select 
    --	'jhub',  13618,	3661.59,	10521794,	342445.47,	649022085,	306226.57,	509404185,	181383,	251725
    */



     '''

In [ ]:
sql_query_2 = f'''
    with params as (
        select
    --         ('01.02.2021' || ' 00:00:00')::timestamp frm,
    --         ('28.02.2021' || ' 23:59:59')::timestamp til
       ( 'DATE_START_replce' || ' 00:00:00')::timestamp    frm 
       ,( 'DATE_END_replce' || ' 23:59:59')::timestamp    til 
    )
    -- select * from params

    , managers as ( /*продакт*/
        select 
        id, 
        concat(last_name, ' ', first_name, ' ', middle_name) name
        from opentech.users 
    --    where appointment in (54, 234) /*категорийный менеджер, заместитель руководителя отдела закупок */
        union all
        select
            0,
            '-- не указан --'
    )

    , bundles as (
        SELECT
            -- a.i i_log
            a1.i  a1_i
            ,a2.i a2_i
            ,a2.id id_order         /* id заказа комплекта */
    --        ,a1.id id_order_item    /* id заказа комплектующей */
            ,s1.reference_id "id комплектующей"
            ,s2.reference_id "id комплекта 1"
            ,s2.id "id комплекта 2"
            ,s2.serial_number 
            ,s1.id "s1_id"
            ,bu.id_bundle
            ,a2.store_id
            ,case when m.id = 0 then m.name
    --        	else '[' || m.name || '|http://opentech.local/user/' || m.id || ']'
    --        	=ГИПЕРССЫЛКА("http://example.microsoft.com/report/budget report.xlsx"; "Щелкните, чтобы просмотреть отчет")
    --        	else '=HYPERLINK("http://opentech.local/user/' || m.id || '"; "' || m.name || '")'
    --          else '=ГИПЕРССЫЛКА("http://opentech.local/user/' || m.id || '"; "' || m.name || '")'
            	else m.name || ' ' || m.id
                	end "продакт"
            ,a2.id_customer  "id_customer Плательщик"
            ,a2.firm order_type
            ,a2.storage_id id_storage
            ,a2.user_login "Кто добавил позицию в заказ"
            ,rp.id "Код справочника позиций"
            ,rp.name name
            ,g.name category
            ,case when b.name is null then 'ПУСТОЕ ЗНАЧЕНИЕ null' else b.name end "Производитель"
    --        ,b.name "Производитель"
            ,s1.price "Продажная цена price"
            ,s1.cost "Расчетная себестоимость cost"
            ,s1.sprice "Себестоимость чистая sprice" -- Закупочная цена по документам

            ,a1.price_net "Цена с учетом скидок и бонус price_net"

            ,a1.price_net - s1.sprice "Прибыль"
            ,(a1.price_net - s1.sprice)/s1.sprice*100 "Наценка, %" 

    --        ,s1.sold state
            ,COALESCE(r.date, checks.created_at)::timestamp realization_date --Дата оформления реализации --Дата/время создания кассовые чеки
            ,ROUND(s1.price * (COALESCE(rnds.value, cnds.value, 20) / (100 + COALESCE(rnds.value, cnds.value, 20))), 2) nds
            ,c.name  "supplier Фирма-поставщик"
            ,u.id manager

            ,s2.date_sold::timestamp "s2_date_sold" --комплект продан 
            ,s1.date_sold::timestamp "s1_date_sold" --его комплектующие списаны 

            ,s2.date_buy::timestamp "s2_date_buy"--Время поступления на склад приходования
            ,s1.date_buy::timestamp "s1_date_buy"

            ,s1.arrive "s1_arrive"
            ,s2.arrive "s2_arrive"

    --        ,a2.in_account_date
    --        ,a2.acc_date
    --        ,a2.fn_date

    --        ,a1.in_account_date
    --        ,a1.acc_date
    --        ,a1.in_account_date

    --        ,ai.expiredate
    --        ,ai.lastpricesupdate
    --        ,ai.tm_created
    --        ,ai.tm_closing
    --        ,ai.tm_placed

    --        ,ai2.expiredate
    --        ,ai2.lastpricesupdate
    --        ,ai2.tm_created
    --        ,ai2.tm_closing
    --        ,ai2.tm_placed
    
        from params _
        cross join opentech.store s1                            /* складская позиция комплектующей */
        left join opentech.accounts a1 on a1.store_id = s1.id        /* заказ комплектующей */
        left join store.bundled_items bu on bu.id_store_item = s1.id
        left join opentech.store s2 on s2.id = bu.id_bundle          /* складская позиция комплекта */
        left join opentech.accounts a2 on a2.store_id = s2.id        /* заказ комплекта */
        /* ндс комплектующей */
        left join orders.realization_items_order_items_bindings bi1 on bi1.id_order_item = a1.i
        left join orders.realization_items i1 on i1.id = bi1.id_realization_item
        left join payments.order_items_locked_by_check_items il1 on il1.id_order_item = a1.i
        left join payments.check_items ci1 on ci1.id = il1.id_check_item_fullpaid
        left join public.nds rnds on rnds.id = i1.id_nds
        left join public.nds cnds on cnds.code = ci1.vat
        /* комплектующая */
        left join opentech.reference_price rp on rp.id = s1.reference_id
        left join opentech.section_new sn on sn.id = rp.section_sootv
        left join managers m on m.id = coalesce(sn.productmanagerid, 0) -- продакт
        left join catalog.plain_section_list g on g.id = sn.goodid
        left join catalog.brands b on b.id = rp.brand_id --,b.name brand
        /* комплект */    
        left join opentech.users u on lower(u.login) = lower(a2.user_login)
        left join orders.realization_items_order_items_bindings bi on bi.id_order_item = a2.i
        left join orders.realization_items i on i.id = bi.id_realization_item
        left join orders.realizations r on r.id = i.id_realization
        left join payments.order_items_locked_by_check_items il on il.id_order_item = a2.i
        left join payments.check_items ci on ci.id = il.id_check_item_fullpaid
        left join payments.checks checks on checks.id = ci.id_check
        left join opentech.supplyitems si on si.id = s2.supplyitemid
        left join opentech.supplies su on su.id = si.supplyid
        left join customers.suppliers sup on sup.id_customer = su.supplierid
        left join opentech.customers c on c.id = coalesce(sup.id_customer_alpha, sup.id_customer)

    --    left join opentech.account_item ai on ai.account_id = a2.id 
    --    left join opentech.account_item ai2 on ai2.account_id = a1.id 

        where 
            a1.state = orders.item_state('BUNDLED')
    --        and b.name ilike 'BaseTech' --BaseTech	484	683681.00	326604.50	48969.34
        	and case when BREND_replce = 0 then true else b.id  = BREND_replce end
    --    	and case when 1 = 0 then true else b.id  = 1392 end
    --        AND s2.sold != store.item_sold('NONE')
    --        AND 
    --        s2.date_sold BETWEEN _.frm AND _.til
    --        AND s2.price > 0

            /*tests*/
    --        and s1.reference_id in (532690
    --       )
    --        and 
    --        a2.id in (6355406,6350260)

    --	order by a2.id, id_bundle

    )
    --select * from bundles 


    , sold as (
    select 	/*1. Производитель|Продано, шт|Продано, руб|Прибыль, руб|Наценка, %*/
        *
    --	"Производитель"
    --	,count(*) "Продано, шт"
    --	,sum("Цена с учетом скидок и бонус price_net") "Продано, руб"
    -- 	,sum("Прибыль") "Прибыль, руб"
    -- 	,round( ((sum("Цена с учетом скидок и бонус price_net") - sum("Себестоимость чистая sprice")) / sum("Себестоимость чистая sprice")) * 100, 0) "Наценка, %"

    from params _
    cross join bundles b
    where 
        s2_date_sold between _.frm and _.til
        /*tests*/
    --	and "Прибыль" <= 0
    --group by "Производитель"
    )
    --select * from sold 


    , completed as ( 
    select 	/*|Скомплектовано, шт|Скомплектовано, руб  */
        *
    --	"Производитель"
    --	,count(*) "Скомплектовано, шт"
    --	,sum("Цена с учетом скидок и бонус price_net") "Скомплектовано, руб"
    from params _
    cross join bundles b
    where 
        (s2_date_sold not between _.frm and _.til or s2_date_sold is null) 
        and 
        s1_date_sold between _.frm and _.til
    --group by "Производитель"
    )
    --select * from completed 


    , bundles_remainder as (
    select /*|Остаток, шт| Остаток, руб*/
        * 
    --	"Производитель"
    --	,count(*) "Остаток, шт"
    --	,sum("Цена с учетом скидок и бонус price_net") "Остаток, руб"
    from params _
    cross join bundles b
    where 
        s2_date_buy < _.til::date --Время поступления на склад приходования --69 942
        and (s2_date_sold IS NULL OR s2_date_sold >= _.til::date) --7 342
        and s2_arrive = ANY(store.arrive_state(ARRAY['NEW', 'OK', 'TRANSIT', 'SELECTED_FOR_TRANSIT'])) -- s.arrive Состояние товара (см. функцию store.arrive_state)
        /*and brand is null*/
    --group by "Производитель"
    )
    --select * from bundles_remainder


    , brand as (
    select 
        case when b.name is null then 'ПУСТОЕ ЗНАЧЕНИЕ null' else b.name end "Производитель"
    --	b.name "Производитель" 
    from catalog.brands b  
    /*where b.name is null*/
    union all 
    select 'ПУСТОЕ ЗНАЧЕНИЕ null' "Производитель" 
    order by 1
    )
    --select * from brand

    --, end_all as (
    --select 
    --	b."Производитель", "Продано, шт", "Продано, руб", "Прибыль, руб", "Наценка, %"
    --	, "Скомплектовано, шт", "Скомплектовано, руб"
    --	,"Остаток, шт" ,"Остаток, руб"
    --from brand b
    --left join sold s on s."Производитель" = b."Производитель"
    --left join completed c on c."Производитель" = b."Производитель"
    --left join bundles_remainder br on br."Производитель" = b."Производитель"
    --where 
    --	"Продано, шт" is not null 
    --	or  
    --	"Скомплектовано, шт" is not null
    --	or  
    --	"Остаток, шт" is not null
    --	
    --order by b."Производитель"  
    --)
    --
    --select * from end_all

    select * from sold
    union all
    select '1000-01-01 00:00:00'
    	,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
    union all
    select * from completed
    union all
    select '1000-01-02 00:00:00'
    	,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
    union all
    select * from bundles_remainder
    
    '''

## Задаем параметры при необходимости

In [ ]:
'''Python OOP set widgets (for set params)'''

# print info (set params)
Prints_().print_set_params()

# ДАТЫ
DATE_START_replce = date.today() - relativedelta(months=1)
DATE_END_replce = date.today()

# значения дат по дефолту устанавливаем которые меняем с помощью пикера! Код ниже ВЫНОСИТЬ НИКУДА НЕ НАДО ИНАЧЕ ПОЕДУТ ЦИФРЫ! поэтому класс ниже через exec!!!
exec(MainVariables().exec_class_DatePicker)

# БРЕНДЫ
df_brand_val = 0 
# значение начальное которое меняем с помощью пикера! Код ниже ВЫНОСИТЬ НИКУДА НЕ НАДО ИНАЧЕ ПОЕДУТ ЦИФРЫ! поэтому класс ниже через exec!!!
exec(MainVariables().exec_class_BrandPicker)


df_for_brand_picker = None
# функция отображения всех заданных здесь наших виджетов
def display_all_set_params_widgets():
    global df_for_brand_picker
    '''display all widgets'''
#     date start, date end
    date_picker1 = DatePicker().dp_start( description = MainVariables().date_picker_description[0], DATE_START_replce = DATE_START_replce )
    date_picker2 = DatePicker().dp_end( description = MainVariables().date_picker_description[1], DATE_END_replce = DATE_END_replce )
#     brand
    df_for_brand_picker = ConnectDatabase().root_database_read_sql_query(MainVariables().sql_query_brand)
    brand_picker2 = BrandPicker().brand_picker(df_for_brand_picker)
    # display( d1 ), display( d2 ), display( bp )
    
    # колонка с датами VBox date_box
    date_box = widgets.VBox([ 
        date_picker1
        ,date_picker2
        ])
    box2 = widgets.VBox([ 
        brand_picker2 
    ])
    all_VBox_widgets = widgets.HBox([ date_box
                                     , box2 
                                    ])
    display(all_VBox_widgets)
#     print(DATE_START_replce, DATE_END_replce )

display_all_set_params_widgets()

In [ ]:
# print(
#     DATE_START_replce, DATE_END_replce, '\n'
#     , df_brand_val, '\n'
#     ,df_for_brand_picker .loc[df_brand_val].to_string()
#      )

In [ ]:
# all_widget_in_this_report_replace_param = {
#     'DATE_START_replce': f'{DATE_START_replce}'
#     ,'DATE_END_replce': f'{DATE_END_replce}'
#     ,'BREND_replce': f'{df_for_brand_picker .loc[df_brand_val][0]}'
#     ,'MANAGER_replce': f'{df_for_brand_picker .loc[df_brand_val][0]}'
#     ,'etc._replce': f'{df_for_brand_picker .loc[df_brand_val][0]}'

#     }
# # all_widget_in_this_report_replace_param

## Жмем кнопку запуска и ждем...

In [ ]:
'''Python OOP run button function widgets'''

all_sql_query_list = [ sql_query_1
                      ,sql_query_2 
                     ]

all_widget_in_this_report_replace_param = '''{
    'DATE_START_replce': f'{DATE_START_replce}'
    ,'DATE_END_replce': f'{DATE_END_replce}'
    ,'BREND_replce': f'{df_for_brand_picker .loc[df_brand_val][0]}'
    ,'MANAGER_replce': f'{df_for_brand_picker .loc[df_brand_val][0]}'
    ,'etc._replce': f'{df_for_brand_picker .loc[df_brand_val][0]}'
    }'''
# eval(all_widget_in_this_report_replace_param)


class MakeOutputFile(FileName, ConnectDatabase, MainVariables):
    def make_name_for_output_file(self, ):
        s3_file_name = self.s3report_file_name( 
            ipynb_file_name .split('.')[0]
            ,getpass_getuser = getpass.getuser()
            ,datetime_now = datetime.datetime.now().strftime("%H:%M:%S")
            ,DATE_START_replce = DATE_START_replce
            ,DATE_END_replce = DATE_END_replce
            ,extension='.xlsx'
            ,params = df_for_brand_picker .loc[df_brand_val][1]
            )
        return s3_file_name
    
    exec(MainVariables().exec_def_sql_replace_params_class_MakeOutputFile)
        
    exec(MainVariables().exec_def_load_sql_into_df_class_MakeOutputFile)
        
    def make_excel_sheets_dict(self, dfs: list) -> dict: 
        try:
            # определяем названия листов в соответствии с данными на эти листы (листы в екселе)
            sheet_name0 = 'Итоговая'
            sheet_name1 = 'Продано Прибыль Наценка'
            sheet_name2 = 'Скомплектовано'
            sheet_name3 = 'Остаток'
            sheets0 = dfs[0]
            sheets1 = dfs[1].astype({'frm': 'str'}).set_index(dfs[1].columns[0]).loc[:spl1].reset_index()
            sheets2 = dfs[1].astype({'frm': 'str'}).set_index(dfs[1].columns[0]).loc[spl1:spl2].reset_index().iloc[1:]
            sheets3 = dfs[1].astype({'frm': 'str'}).set_index(dfs[1].columns[0]).loc[spl2:].reset_index().iloc[1:]
            # СЛОВАРЬ названия листов и их данных 
            excel_sheets = {
                f'1 - {sheet_name0}' : sheets0, 
                f'2 - {sheet_name1}' : sheets1, 
                f'3 - {sheet_name2}' : sheets2, 
                f'4 - {sheet_name3}' : sheets3, 
                }
            return excel_sheets
        except Exception as exc:
                print( Prints_().print_err_make_excel_sheets_dict(), exc )

# if we have changeable parameters, replace their into the sql query and print! for exemple for test on DBeaver 
# print( MakeOutputFile().sql_replace_params(all_sql_query_list, all_widget_in_this_report_replace_param)[0]   )

# 
exec(MainVariables().exec_class_RunFunction)
#     
exec(MainVariables().exec_class_RunClearButtons)
RunClearButtons().display_run_clear_button_widgets()   

pd.options.display.float_format = '{:.0f}'.format 
pd.set_option('max_rows', 500) 


In [ ]:
# print( eval(all_widget_in_this_report_replace_param) )

In [ ]:
### if we have changeable parameters, replace their into the sql query and print! for exemple for test on DBeaver 
# print( MakeOutputFile().make_name_for_output_file() ) 
# print( MakeOutputFile().sql_replace_params(all_sql_query_list,  eval(all_widget_in_this_report_replace_param))[0]   )
# df_for_brand_picker .loc[df_brand_val][0]

# Конец отчета

### Заметки!

In [ ]:
# info for users
try:
    Prints_().print_info4users( )
except:
    pass
